In [2]:
import os

In [3]:
OPENAI_API_KEY = ''
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [4]:
from langchain_community.vectorstores import Chroma
NOMBRE_INDICE_CHROMA = "../databases/embeddings-public"

In [5]:
from langchain_community.embeddings import SentenceTransformerEmbeddings

embeddings_st = SentenceTransformerEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

# Otro modelo en español que podríamos usar es "symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli"

/home/egarces/chatbot/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
vectorstore_chroma = Chroma(
    persist_directory=NOMBRE_INDICE_CHROMA,
    embedding_function=embeddings_st
)

In [14]:
print(vectorstore_chroma)

In [15]:
query = "Cual es el parametro para configurar el nuevo reporte Pivot?"

docs = vectorstore_chroma.similarity_search_with_score(query, k=3)
docs

[(Document(page_content='(BETA): Nuevo tipo de reportes PIVOT , para configurarlos deje el parámetro UseNewPivotReportVersion en true, y las demás configuraciones habituales en las tablas defaultreport (Por ahora solo es posible mostrar la data, Expórtala a Excel y generar graficas de barra) Pruebas y aplicabilidad. Todas las apps.', metadata={'source': 110816}),
  5.054394721984863),
 (Document(page_content='Es posible que en reportes tipo PIVOT se pueda configurar de tal forma que se puedan abrir los elementos de forma directa, en el último nivel de detalle', metadata={'source': 33608}),
  7.683204174041748),
 (Document(page_content='Se ajustaron los estilos del nuevo reporte pivot, manteniendo los estilos de sense.Pruebas y aplicabilidad. Todas las apps.', metadata={'source': 112932}),
  8.664474487304688)]

In [8]:
retriever_chroma = vectorstore_chroma.as_retriever(
    search_kwargs={'k': 3}
)

In [9]:
#pip install -U langchain-openai
#from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI

llm_openIA = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

In [10]:
from langchain.chains import RetrievalQAWithSourcesChain

qa_chain_with_sources = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm_openIA,
    chain_type="stuff",
    retriever=retriever_chroma
)


In [11]:
query = "Cual es el parametro para configurar el nuevo reporte Pivot?"
respuesta = qa_chain_with_sources.invoke(query)
respuesta

{'question': 'Cual es el parametro para configurar el nuevo reporte Pivot?',
 'answer': 'El parámetro para configurar el nuevo reporte Pivot es UseNewPivotReportVersion en true.\n',
 'sources': '110816'}

In [12]:
from langchain.chains import RetrievalQAWithSourcesChain, ConversationalRetrievalChain, RetrievalQA    
qa_chain = RetrievalQA.from_chain_type(
        llm=llm_openIA, chain_type="stuff", retriever=retriever_chroma
    )


In [15]:
retriever_chroma

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7fa9b0825c30>, search_kwargs={'k': 3})

In [13]:
query = "Cual es el parametro para configurar el nuevo reporte Pivot?"
respuesta = qa_chain.invoke(query)
respuesta

{'query': 'Cual es el parametro para configurar el nuevo reporte Pivot?',
 'result': 'El parámetro para configurar el nuevo reporte Pivot es "UseNewPivotReportVersion" y debe establecerse en true.'}

In [17]:
query = "¿Cuál es el río más largo del mundo?"
results = retriever_chroma.search(query)
results

AttributeError: 'VectorStoreRetriever' object has no attribute 'search'